In [25]:
import torch

import pandas as pd
import numpy as np
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests 
import itertools
import spotlight

In [26]:
with open('templates.json') as f:
    templates = json.load(f)

with open('data/lc-quad/test/input.txt') as f:
    question = f.read().strip().split('\n')
    
with open('data/lc-quad/test/input.pos') as f:
    pos = f.read().strip().split('\n')
    
with open('data/lc-quad/test/output.txt') as f:
    output = f.read().strip().split('\n')

In [55]:
# Sorts based on descending order of values
def sort_dict_by_values(dictionary):
    keys = []
    for key, value in sorted(dictionary.items(), key=lambda item: (item[1], item[0]), reverse=True):
        keys.append(key)
    return keys

def get_earl_entities(query):
    THRESHOLD = 0.0001
    MAX_ENTITIES_PER_TYPE = 4
    
    response = requests.post('http://sda.tech/earl/api/processQuery', 
                             json={"nlquery": query, "pagerankflag": False})
    
    json_response = json.loads(response.text)
    r_dict = {}
    c_dict = {}
    p_dict = {}
    
    reranked_lists = json_response['rerankedlists']
    for key in reranked_lists.keys():
        for result in reranked_lists[key]:
            if result[0] < THRESHOLD:
                continue                
            
            if result[1].startswith('http://dbpedia.org/resource/'):
                r_dict[result[1]] = result[0]
            elif result[1].split('/')[-1][0].isupper():
                c_dict[result[1]] = result[0]
            else:
                p_dict[result[1]] = result[0]
        
    return {
        'r': sort_dict_by_values(r_dict)[:MAX_ENTITIES_PER_TYPE], 
        'p': sort_dict_by_values(p_dict)[:MAX_ENTITIES_PER_TYPE], 
        'c': sort_dict_by_values(c_dict)[:MAX_ENTITIES_PER_TYPE]
    }

In [28]:
def get_spotlight_entities(query):
    annotations = spotlight.annotate('http://model.dbpedia-spotlight.org/en/annotate',
                                     query,
                                     confidence=0.4, support=1)
    print(annotations)

In [29]:
def get_lookup_entities(query):
    response = requests.get('http://lookup.dbpedia.org/api/search/KeywordSearch?QueryString=%s'.format(query),
                           headers={'Accept': 'application/json'})
    
    print(response.text)

In [30]:
def make_sparql_query(query, return_var):
    sparql = SPARQLWrapper("http://akswnc9.aksw.uni-leipzig.de/dbpedia/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    if return_var == 'boolean':
        if 'boolean' in results:            
            return results['boolean']
        else:
            return None
    
    output = []
    for result in results["results"]["bindings"]:
        if return_var in result:
            output.append(result[return_var]['value'])
    return output

In [56]:
def get_answer(question, template):
    output = []
    sparql_query = ''
    
    entities = get_earl_entities(question)
    slots = {}
    for slot in template['slots']:
        slots[slot] = entities[slot[0]]        
    
    # This means something probably went wrong and no predicates or resources were detected for the query
    if len(slots['p']) == 0 or len(slots['r']) == 0:
        return output, sparql_query
    
    ranges = []
    slot_keys = list(slots.keys())
    slot_len = len(slot_keys)
    for slot in slot_keys:
        ranges.append(range(len(slots[slot]))) 
        
    for i in itertools.product(*ranges):
        if slot_keys[0] == 'p' and slot_keys[1] == 'p2' and slots['p'][i[0]] == slots['p2'][i[1]]:
            continue
        if slot_keys[-2] == 'r' and slot_keys[-1] == 'r2' and slots['r'][i[-2]] == slots['r2'][i[-1]]:
            continue
            
        sparql_query = template['sparql']                
        for index in range(slot_len):
            sparql_query = sparql_query.replace('<' + slot_keys[index] + '>', '<' + slots[slot_keys[index]][i[index]] + '>')

        output = make_sparql_query(sparql_query, template['return'])        
        if template['return'] == 'boolean':
            if output == None:
                output = []
            else:
                output = [output]
        elif len(output) > 0:
            break
    
    return output, sparql_query

In [53]:
with open('data/qald/qald.json') as f:
    qald_questions = json.load(f)

In [58]:
accuracy = 0
for index in range(len(qald_questions)):
    question = qald_questions[index]    

    if question['answertype'] == 'resource' and question['onlydbo'] == True:
        question_string = question['question'][0]['string']
        predictions = question['predictions'].split(',')
        print(question_string)
        
        answers, query = get_answer(question_string, templates[predictions[0]])
        
        if len(answers) == 0:
            answers, query = get_answer(question_string, templates[predictions[1]])
        
        if len(answers) == 0:
            pass
        else:
            print('ANSWERS: ', answers, '\n')
            print('QUERY: ', query, '\n')
            accuracy += 1
        
        print('ACTUAL ANSWERS:', question['answers'][0]['results']['bindings'], '\n')
        print('ACTUAL QUERY:', question['query']['sparql'], '\n')            
        print('\n\n\n\n')        


List all the musicals with music by Elton John.
ANSWERS:  ['0'] 

QUERY:  SELECT (COUNT(DISTINCT ?uri) as ?count) WHERE { ?uri <http://dbpedia.org/ontology/musicComposer> <http://dbpedia.org/resource/Electricity_(Elton_John_song)> . OPTIONAL { ?uri rdf:type <http://dbpedia.org/ontology/Musical> } } 

ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/The_Lion_King_(musical)'}}, {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Lestat_(musical)'}}, {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Billy_Elliot_the_Musical'}}, {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Aida_(musical)'}}] 

ACTUAL QUERY: PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX res: <http://dbpedia.org/resource/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
SELECT DISTINCT ?uri
WHERE { 
        ?uri rdf:type dbo:Musical .
        ?uri dbo:musicBy res:Elton_John .
} 






Who was the wife of U.S. president Lincoln?
ANSWERS:  ['ht

ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Ken_Follett'}}] 

ACTUAL QUERY: PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX res: <http://dbpedia.org/resource/>
SELECT DISTINCT ?uri 
WHERE {
	res:The_Pillars_of_the_Earth dbo:author ?uri .
} 






Who discovered Ceres?
ANSWERS:  ['http://dbpedia.org/resource/Panaad_Park_and_Stadium'] 

QUERY:  SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Ceres_F.C.> <http://dbpedia.org/ontology/ground> ?uri . } 

ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Giuseppe_Piazzi'}}] 

ACTUAL QUERY: SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Ceres_(dwarf_planet)> <http://dbpedia.org/ontology/discoverer> ?uri . } 






Which river does the Brooklyn Bridge cross?
ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/East_River'}}] 

ACTUAL QUERY: PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX res: <http://dbpedia.org/resource/>
SELECT D

ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Seth_MacFarlane'}}] 

ACTUAL QUERY: SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Family_Guy> <http://dbpedia.org/ontology/creator> ?uri . }   






What is the official language of Suriname?
ANSWERS:  ['http://dbpedia.org/resource/Dutch_language'] 

QUERY:  SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Suriname> <http://dbpedia.org/ontology/officialLanguage> ?uri . } 

ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Dutch_language'}}] 

ACTUAL QUERY: PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX res: <http://dbpedia.org/resource/>
SELECT DISTINCT ?uri
WHERE { 
        res:Suriname dbo:officialLanguage ?uri .
} 






Who wrote the song Hotel California?
ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Don_Felder'}}, {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Glenn_Frey'}}, {'uri': {'type': 'uri', 'va

ANSWERS:  ['http://dbpedia.org/resource/Sydney'] 

QUERY:  SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Australia> <http://dbpedia.org/ontology/largestCity> ?uri . } 

ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Sydney'}}] 

ACTUAL QUERY: PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX res: <http://dbpedia.org/resource/>
SELECT DISTINCT ?uri 
WHERE {
	res:Australia dbo:largestCity ?uri .
} 






Who is the mayor of Tel Aviv?
ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Ron_Huldai'}}] 

ACTUAL QUERY: PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX res: <http://dbpedia.org/resource/>
SELECT DISTINCT ?uri
WHERE { 
        res:Tel_Aviv dbo:leaderName ?uri .
} 






List all episodes of the first season of the HBO television series The Sopranos.
ANSWERS:  ['86'] 

QUERY:  SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/The_Sopranos> <http://dbpedia.org/ontology/numberOfEpisodes> ?uri . } 

ACTU

ANSWERS:  [False] 

QUERY:  ASK WHERE { <http://dbpedia.org/resource/Guomao,_Beijing> <http://dbpedia.org/ontology/established> <http://dbpedia.org/resource/That's_Beijing> } 

ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Lenovo'}}] 

ACTUAL QUERY: SELECT DISTINCT ?uri WHERE { ?uri a <http://dbpedia.org/ontology/Company> . ?uri <http://dbpedia.org/ontology/industry> <http://dbpedia.org/resource/Electronics> . ?uri <http://dbpedia.org/ontology/foundationPlace> <http://dbpedia.org/resource/Beijing> . } 






Give me all video games published by Mean Hamster Software.
ANSWERS:  [False] 

QUERY:  ASK WHERE { <http://dbpedia.org/resource/Software> <http://dbpedia.org/ontology/SpaceStation/volume> <http://dbpedia.org/resource/Hamster_Corporation> } 

ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Crossbow_(video_game)'}}, {'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Riven'}}] 

ACTUAL QUERY: PREFIX dbo: <http

ANSWERS:  ['0'] 

QUERY:  SELECT (COUNT(DISTINCT ?uri) as ?count) WHERE { ?x <http://dbpedia.org/ontology/creator> <http://dbpedia.org/resource/Count_Dracula> . ?uri <http://dbpedia.org/ontology/motto> ?x . ?uri rdf:type <http://dbpedia.org/ontology/Fungus> } 

ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': '1912-04-20'}}] 

ACTUAL QUERY: SELECT DISTINCT ?date WHERE {  <http://dbpedia.org/resource/Count_Dracula> <http://dbpedia.org/ontology/creator> ?x .  ?x <http://dbpedia.org/ontology/deathDate> ?date . }  






Which country does the creator of Miffy come from?
ACTUAL ANSWERS: [{'uri': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Netherlands'}}] 

ACTUAL QUERY: PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX res: <http://dbpedia.org/resource/>
SELECT DISTINCT ?uri 
WHERE {
	res:Miffy dbo:creator ?x .
	?x dbo:nationality ?uri .
} 






What form of government does Russia have?
ANSWERS:  ['http://dbpedia.org/resource/Russian_language'] 

QUERY:  SELECT DISTINCT ?uri WH

KeyboardInterrupt: 